In [1]:
from collections import deque
import copy
import graphviz
from pickle import NONE
import string

In [2]:
class drawDFA:
    def __init__(self, DFA):
        self.DFA = DFA
        self.dfa = graphviz.Digraph("dfa", format='png') 
        self.dfa.attr(rankdir='LR', size='9')
        self.setStates()
        self.setTransitions()
        self.draw()
    def setStates(self):
        self.dfa.node("", style="invis")
        self.dfa.attr('node', shape='circle')
        self.dfa.node(str(self.DFA.initial_state))
        self.dfa.attr('node', shape='doublecircle')

        for state in self.DFA.accepting_states:
            self.dfa.node(str(state))
    def setTransitions(self):
        self.dfa.edge("", str(self.DFA.initial_state))
        self.dfa.attr('node', shape='circle')
        for state in self.DFA.transitions:
            for substates in self.DFA.transitions[state]:
                for con in self.DFA.transitions[state][substates]:
                    self.dfa.edge(str(state), str(con), label=str(substates))

    def draw(self):
        self.dfa.render(directory='output').replace('\\', '/')

In [3]:
class Finite_automata:
    def __init__(self, alphabet, states, initial_state, accepting_states, transitions = None):
        self.alphabet = alphabet
        self.states = states
        self.initial_state = initial_state
        self.accepting_states = accepting_states
        self.transitions = transitions

    def give_info(self):
        print("----------------- Automata -----------------")
        print("The alphabet work with was: %s".format(self.alphabet))
        print("The states of the automata was: %s".format(self.states))
        print("The initial states is: %s".format(self.initial_state))
        print("The accepting_states are: %s".format(self.accepting_states))
        print("The transition function is given by:")
        print(self.transitions)

    def copy_transitions(self, transitions):
        self.transitions = transitions
    def __str__(self) -> str:
        return ("Alphabet: " + str(self.alphabet) + "\n" +
                "States: " + str(self.states) + "\n" +
                "Initial State: " + str(self.initial_state) + "\n" +
                "Accepting States: " + str(self.accepting_states) + "\n" +
                "Transition Function: " + str(self.transitions))

In [4]:
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None
    def get_value(self):
        return self.value

In [5]:
class Stack:
    def __init__(self, initial = None):
        if initial == None:
            self.items = deque()
        else:
            self.items = deque(initial)
    def __len__(self):
        return len(self.items)
    def __contains__(self, item):
        return item in self._items
    def pop(self):
        try:
            return self.items.pop()
        except IndexError:
            raise IndexError("dequeue from an empty queue") from None
    def popleft(self):
        try:
            return self.items.popleft()
        except IndexError:
            raise IndexError("dequeue from an empty queue") from None
    def push(self, value):
        if not hasattr(value, "__len__"):
            self.items.append(value)
        else:
            for elm in value:
                self.items.append(elm)
    def get_size(self):
        return self.__len__();
    def is_empty(self):
        return self.__len__() == 0
    def peek(self):
        if self.is_empty():
            raise Exception("It's empty");
        return self.items[len(self.items) - 1];
    def __str__(self):
        to_print = "["
        for elm in self.items:
            to_print += str(elm) + ", "
        to_print += "]"
        return to_print;


In [6]:
class SubsetConstruction:
    def __init__(self, nfa: Finite_automata):
        self.nfa = nfa
    def calculeEpsClosure(self, T):
        if not hasattr(T, "__len__"):
            T = [T]
        stack = Stack(T)
        epsClosure = T
        while len(stack) != 0:
            t = stack.pop()
            if "" in self.nfa.transitions[t]:
                for u in self.nfa.transitions[t][""]:
                    if u not in epsClosure:
                        epsClosure.append(u)
                        stack.push(u)
        return epsClosure
    def move(self, T, a):
        states = set()
        for t in T:
            if a in self.nfa.transitions[t]:
                states.update(self.nfa.transitions[t][a])
        return list(states)
    def setAsInt(self, list):
        rep = [0]*len(self.nfa.states)
        for i in list:
            rep[i] = 1
        return int("".join(map(str, rep)),2)
    def intAsSet(self, num):
        setRep = []
        binRep = "{0:b}".format(num).zfill(len(self.nfa.states))
        for i in range(len(binRep)):
            if binRep[i] == "1":
                setRep.append(i)
        return setRep
    def prettyStates(self, dfa):
        newKeys = {}
        index = 0
        for state in dfa.transitions:
            dfa.states.index(state)
            newKeys[state] = "q" + str(index)
            index += 1
        for i in range(len(dfa.states)):
            dfa.states[i] = newKeys[dfa.states[i]]
        dfa.initial_state = newKeys[dfa.initial_state]
        for i in range(len(dfa.accepting_states)):
            dfa.accepting_states[i] = newKeys[dfa.accepting_states[i]]
        newTransitions = {}
        for rule in dfa.transitions:
            newSubDic = {}
            for subrule in dfa.transitions[rule]:
                newTrans = []
                for state in dfa.transitions[rule][subrule]:
                    newTrans.append(newKeys[state])
                newSubDic[subrule] = newTrans
            newTransitions[newKeys[rule]] = newSubDic
        return Finite_automata(self.nfa.alphabet, dfa.states, dfa.initial_state, dfa.accepting_states, newTransitions)
    def isAccepting(self, list):
        for state in list:
            if state in self.nfa.accepting_states:
                return True
        return False
    def calculeSubsetConstruction(self):
        unmarked = Stack()
        dStates = [self.setAsInt(self.calculeEpsClosure(self.nfa.initial_state))]
        startState = dStates[0]
        acceptingStates = []
        unmarked.push(0)
        dTran = {}
        while len(unmarked) > 0:
            TN = unmarked.popleft()
            T = self.intAsSet(dStates[TN])
            for a in self.nfa.alphabet:
                U = self.calculeEpsClosure(self.move(T, a))
                aliasU = self.setAsInt(U)
                if aliasU not in dStates:
                    if self.isAccepting(U):
                        acceptingStates.append(aliasU)
                    dStates.append(aliasU)
                    unmarked.push(len(dStates) - 1)
                if not dStates[TN] in dTran:
                    dTran[dStates[TN]]={}
                if not a in dTran[dStates[TN]]:
                    dTran[dStates[TN]][a]=[]
                dTran[dStates[TN]][a].append(self.setAsInt(U))
        return self.prettyStates(Finite_automata(self.nfa.alphabet, dStates, startState, acceptingStates, dTran))

In [7]:
def add_concatenation_operation(regex, alphabet):
    temp_regex = list(regex)
    new_regex = ""
    for i in range(len(temp_regex)-1):
        if ((temp_regex[i] in alphabet or temp_regex[i]==')') and (temp_regex[i+1] in alphabet or temp_regex[i+1] == '(' )):
            new_regex = new_regex + temp_regex[i] + ".";
        elif ((temp_regex[i]==')' or temp_regex[i]=='*') and (temp_regex[i+1] in alphabet or temp_regex[i+1]=='(')):
            new_regex = new_regex + temp_regex[i] + ".";
        else:
            new_regex += temp_regex[i]
    new_regex += temp_regex[-1]
    return new_regex

In [8]:
def has_higher_precedence(first_operator, second_operator):
    return (first_operator <= second_operator)

In [9]:
def infix_to_postfix(regex, alphabet, operations):
    operation_order = Stack();
    post_string = ""
    for i in range(len(regex)):
        if regex[i] in alphabet:
            post_string += regex[i]
        elif regex[i] in operations:
            while (not operation_order.is_empty() and has_higher_precedence(operation_order.peek(), regex[i])) and (operation_order.peek()!='('):
                post_string += operation_order.pop()
            operation_order.push(regex[i])
        elif regex[i]=='(':
            operation_order.push('(')
        elif regex[i]==')':
            while (not operation_order.is_empty() and operation_order.peek()!='('):
                post_string += operation_order.pop()
            operation_order.pop()
    while (not operation_order.is_empty()):
        post_string += operation_order.pop()
    return post_string

In [10]:
def regex_to_nfda(regex):
    alphabet = string.ascii_uppercase + string.ascii_lowercase + string.digits + '_ =';
    operations = "|.*";
    
    regex = remove_hyphens(regex)
    regex = add_concatenation_operation(regex, alphabet)
    print(regex)
    postfix_regex = infix_to_postfix(regex, alphabet, operations)
    print(postfix_regex)

    diff_symbols = []
    for char in postfix_regex:
        if (char in alphabet and char not in diff_symbols):
            diff_symbols.append(char)
    diff_symbols.sort()
    columns = len(diff_symbols) + 1

    states = []
    initial_states = []
    accepting_states = []
    operands = Stack()
    transitions = {}
    rows  = 0
    for char in postfix_regex:
        if char == '.':
            transitions[accepting_states.pop(-2)][-1].append(initial_states.pop())
            second_operand = operands.pop()
            first_operand = operands.pop()
            operands.push(first_operand+"."+second_operand)
        else:
            initial_state = "q"+str(rows)
            states.append(initial_state)
            rows+=1
            transitions[initial_state] = [None]*columns
            transitions[initial_state][-1] = []
            accepting_state = "q"+str(rows)
            states.append(accepting_state)
            transitions[accepting_state] = [None]*columns
            transitions[accepting_state][-1] = []
            rows+=1
            if char in alphabet:
                transitions[initial_state][diff_symbols.index(char)] = accepting_state
                operands.push(char)
            if char == '|':
                temp_accepting_states = []
                temp_accepting_states.append(accepting_states.pop())
                temp_accepting_states.append(accepting_states.pop())
                for current_state in temp_accepting_states:
                    transitions[current_state][-1].append(accepting_state)

                transitions[initial_state][-1].append(initial_states.pop())
                transitions[initial_state][-1].append(initial_states.pop())

                second_operand = operands.pop()
                first_operand = operands.pop()
                operands.push("("+first_operand + "|" + second_operand+")")
            if char == '*':
                transitions[initial_state][-1].append(initial_states.pop())
                transitions[initial_state][-1].append(accepting_state)

                transitions[accepting_states.pop()][-1].append(accepting_state)

                transitions[accepting_state][-1].append(initial_state)

                operand = operands.pop()
                operands.push(operand+"*")

            initial_states.append(initial_state)
            accepting_states.append(accepting_state)

    nfda = Finite_automata(alphabet, states, "".join(initial_states), accepting_states)
    nfda.copy_transitions(transitions)
    
    return nfda

In [11]:
def remove_hyphens(regex):
    while '-' in regex:
        index = regex.index('-')
        replacement = ""
        if regex[index-1] in string.ascii_letters:
            first_char_index = string.ascii_letters.index(regex[index-1])
            last_char_index = string.ascii_letters.index(regex[index+1])
            for i in range(first_char_index, last_char_index):
                replacement += string.ascii_letters[i] + '|'
            #replacement += string.ascii_letters[last_char_index]
            regex = regex[:index-1] + replacement + regex[index+1:]
        if regex[index-1] in string.digits:
            first_char_index = string.digits.index(regex[index-1])
            last_char_index = string.digits.index(regex[index+1])
            for i in range(first_char_index, last_char_index):
                replacement += string.digits[i] + '|'
            #replacement += string.digits[last_char_index]
            regex = regex[:index-1] + replacement + regex[index+1:]
        regex = regex.replace('[','(')
        regex = regex.replace(']',')')
    return regex

In [16]:
def main():
    current_nfda = regex_to_nfda('for|FOR')
    trans = SubsetConstruction(current_nfda)
    dfa = trans.calculeSubsetConstruction()
    print(dfa)
    draw = drawDFA.drawDFA(dfa)

In [18]:
if __name__ == "__main__":
    main()

f.o.r|F.O.R
fo.r.FO.R.|


KeyError: '2'